In [2]:
pip install xgboost

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from tensorflow.keras.utils import to_categorical

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,LSTM,Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
sns.set()

In [4]:
# Loading  datasets
ranking = pd.read_excel('fifa_ranking.xlsx')
results = pd.read_csv('results.csv')
fixtures = pd.read_excel('fixtures.xlsx')

In [5]:
results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
43081,2021-12-15,Tunisia,Egypt,NaN,NaN,Arab Cup,Doha,Qatar,True
43082,2021-12-15,Qatar,Algeria,NaN,NaN,Arab Cup,Doha,Qatar,False
43083,2021-12-18,NaN,NaN,NaN,NaN,Arab Cup,Doha,Qatar,NaN
43084,2021-12-18,NaN,NaN,NaN,NaN,Arab Cup,Al Khor,Qatar,NaN
43085,2021-12-18,United States,Bosnia and Herzegovina,NaN,NaN,Friendly,Los Angeles,United States,False


In [6]:
ranking.head()


,Positions,Team,Total Points
0,20,Senegal,1561.68
1,28,Morocco,1529.93
2,29,Algeria,1516.51
3,30,Tunisia,1512.72
4,36,Nigeria,1478.78


In [7]:
results.dropna(subset = ["home_score"], inplace=True)

In [8]:
results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
43076,2021-12-10,Tunisia,Oman,2.0,1.0,Arab Cup,Al Rayyan,Qatar,True
43077,2021-12-10,Qatar,United Arab Emirates,5.0,0.0,Arab Cup,Al Khor,Qatar,False
43078,2021-12-11,Egypt,Jordan,3.0,1.0,Arab Cup,Al Wakrah,Qatar,True
43079,2021-12-11,Morocco,Algeria,2.0,2.0,Arab Cup,Doha,Qatar,True
43080,2021-12-11,El Salvador,Chile,0.0,1.0,Friendly,Los Angeles,United States,True


In [9]:
#Adding goal difference and establishing who is the winner 
winner = []
for i in results.index.tolist():
    
    if results ['home_score'][i] > results['away_score'][i]:
        winner.append(results['home_team'][i])
    elif results['home_score'][i] < results ['away_score'][i]:
        winner.append(results['away_team'][i])
    else:
        winner.append('Draw')
results['winning_team'] = winner

#adding goal difference column
results['goal_difference'] = np.absolute(results['home_score'] - results['away_score'])



In [10]:
results.tail(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team,goal_difference
43071,2021-12-07,Morocco,Saudi Arabia,1.0,0.0,Arab Cup,Doha,Qatar,True,Morocco,1.0
43072,2021-12-07,Algeria,Egypt,1.0,1.0,Arab Cup,Al Wakrah,Qatar,True,Draw,0.0
43073,2021-12-07,Lebanon,Sudan,1.0,0.0,Arab Cup,Al Rayyan,Qatar,True,Lebanon,1.0
43074,2021-12-08,Mexico,Chile,2.0,2.0,Friendly,Austin,United States,True,Draw,0.0
43075,2021-12-09,Tanzania,Uganda,0.0,2.0,Friendly,Dar es Salaam,Tanzania,False,Uganda,2.0
43076,2021-12-10,Tunisia,Oman,2.0,1.0,Arab Cup,Al Rayyan,Qatar,True,Tunisia,1.0
43077,2021-12-10,Qatar,United Arab Emirates,5.0,0.0,Arab Cup,Al Khor,Qatar,False,Qatar,5.0
43078,2021-12-11,Egypt,Jordan,3.0,1.0,Arab Cup,Al Wakrah,Qatar,True,Egypt,2.0
43079,2021-12-11,Morocco,Algeria,2.0,2.0,Arab Cup,Doha,Qatar,True,Draw,0.0
43080,2021-12-11,El Salvador,Chile,0.0,1.0,Friendly,Los Angeles,United States,True,Chile,1.0


In [11]:
africacup_teams = ['Cameroon', 'Cape Verde', 'Ethiopia', 'Burkina Faso',
                  'Guinea', 'Egypt', 'Morocco', 'Nigeria',
                  'Senegal', 'Tunisia', 'Malawi', 'Zimbabwe',
                  'Comoros', 'Gabon', 'Ghana', 'Guinea-Bissau',
                  'Sudan', 'Algeria', "Ivory Coast", 'Equatorial Guinea',
                  'Sierra Leone', 'Gambia', 'Mali', 'Mauritania']


In [12]:
df_teams_home = results[results['home_team'].isin(africacup_teams)]
df_teams_away = results[results['away_team'].isin(africacup_teams)]
df_teams = pd.concat((df_teams_home, df_teams_away))
df_teams.drop_duplicates()
df_teams.count()

date               10019
home_team          10019
away_team          10019
home_score         10019
away_score         10019
tournament         10019
city               10019
country            10019
neutral            10019
winning_team       10019
goal_difference    10019
dtype: int64

In [13]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10019 entries, 809 to 43079
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             10019 non-null  object 
 1   home_team        10019 non-null  object 
 2   away_team        10019 non-null  object 
 3   home_score       10019 non-null  float64
 4   away_score       10019 non-null  float64
 5   tournament       10019 non-null  object 
 6   city             10019 non-null  object 
 7   country          10019 non-null  object 
 8   neutral          10019 non-null  object 
 9   winning_team     10019 non-null  object 
 10  goal_difference  10019 non-null  float64
dtypes: float64(3), object(8)
memory usage: 939.3+ KB


In [14]:
year = []
for row in df_teams['date']:
    year.append(int(row[:4]))
df_teams['match_year'] = year
df_teams_1960 = df_teams[df_teams.match_year >= 1980]
df_teams_1960.head()

#dropping columns that wll not affect matchoutcomes
df_teams_1960 = df_teams.drop(['date', 'home_score', 'away_score', 'tournament', 'city', 'country',
                               'goal_difference', 'match_year', 'neutral'], axis = 1)

df_teams_1960.head()



,home_team,away_team,winning_team
809,Egypt,Lithuania,Egypt
1463,Egypt,Hungary,Draw
1661,Egypt,Israel,Egypt
1695,Egypt,Hungary,Hungary
1894,Egypt,Greece,Egypt


In [15]:
df_teams_1960 = df_teams_1960.reset_index(drop=True)
df_teams_1960.loc[df_teams_1960.winning_team == df_teams_1960.home_team,'winning_team']=2
df_teams_1960.loc[df_teams_1960.winning_team == 'Draw', 'winning_team']=1
df_teams_1960.loc[df_teams_1960.winning_team == df_teams_1960.away_team, 'winning_team']=0

df_teams_1960.head()

,home_team,away_team,winning_team
0,Egypt,Lithuania,2
1,Egypt,Hungary,1
2,Egypt,Israel,2
3,Egypt,Hungary,0
4,Egypt,Greece,2


In [16]:
#convert home team and away team from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_1960, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Separate X and y sets
X = final.drop(['winning_team'], axis=1)
y = final["winning_team"]
y = y.astype('int')

# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [17]:
final.head(10)

,winning_team,home_team_Albania,home_team_Algeria,home_team_Andalusia,home_team_Andorra,home_team_Angola,home_team_Argentina,home_team_Armenia,home_team_Australia,home_team_Austria,home_team_Bahrain,home_team_Bangladesh,home_team_Basque Country,home_team_Belarus,home_team_Belgium,home_team_Benin,home_team_Bolivia,home_team_Bosnia and Herzegovina,home_team_Botswana,home_team_Brazil,home_team_Brittany,home_team_Bulgaria,home_team_Burkina Faso,home_team_Burundi,home_team_Cameroon,home_team_Canada,home_team_Cape Verde,home_team_Catalonia,home_team_Central African Republic,home_team_Chad,home_team_Chile,home_team_China PR,home_team_Colombia,home_team_Comoros,home_team_Congo,home_team_Corsica,home_team_Costa Rica,home_team_Croatia,home_team_Cuba,home_team_Czech Republic,...,away_team_Rwanda,away_team_Réunion,away_team_Saudi Arabia,away_team_Scotland,away_team_Senegal,away_team_Serbia,away_team_Seychelles,away_team_Sierra Leone,away_team_Slovakia,away_team_Slovenia,away_team_Somalia,away_team_South Africa,away_team_South Korea,away_team_South Sudan,away_team_Spain,away_team_Sudan,away_team_Suriname,away_team_Sweden,away_team_Switzerland,away_team_Syria,away_team_São Tomé and Príncipe,away_team_Tanzania,away_team_Thailand,away_team_Togo,away_team_Trinidad and Tobago,away_team_Tunisia,away_team_Turkey,away_team_Uganda,away_team_United Arab Emirates,away_team_United States,away_team_Uruguay,away_team_Uzbekistan,away_team_Venezuela,away_team_Wales,away_team_Yemen,away_team_Yemen DPR,away_team_Yugoslavia,away_team_Zambia,away_team_Zanzibar,away_team_Zimbabwe
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


**Logistic Regression**

In [18]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
score = logreg.score(X_train, y_train)
score2 = logreg.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.564
Test set accuracy:  0.536


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


**Naive Bayes**

In [19]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
score = gnb.score(X_train, y_train)
score2 = gnb.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.327
Test set accuracy:  0.302


**XGBClassifier**

In [20]:

# instantiate a XGBClassifier model
xgb_clf = XGBClassifier(num_class=3,learning_rate=0.1,max_depth=10,eval_metric='mlogloss')
# train the model using X_train
xgb_clf.fit(X_train, y_train)
# make class predictions for X_train and X_test
score = xgb_clf.score(X_train, y_train)
score2 = xgb_clf.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))


Training set accuracy:  0.575
Test set accuracy:  0.530


In [21]:
# make class predictions for X_test
y_pred_class = xgb_clf.predict(X_test)
# calculate accuracy of class predictions
accuracy = metrics.accuracy_score(y_test, y_pred_class)
# the confusion matrix
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_class)

In [22]:
print(confusion_matrix)
print(accuracy)

[[ 136   89  484]
 [  64  109  647]
 [  67   63 1347]]
0.5296074517631404


**DNN**

In [23]:
print(X_train.shape)
print(y_train.shape)

(7013, 312)
(7013,)


In [24]:

y_t = to_categorical(y_train,num_classes=3)
y_ts = to_categorical(y_test,num_classes=3)

In [25]:
print(y_t.shape)
print(y_ts.shape)

(7013, 3)
(3006, 3)


In [27]:

# Creating a Neural Network Model


# having 19 neuron is based on the number of available features
model = Sequential()
model.add(Dense(1024,input_dim= X_train.shape[1],activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))

model.add(Dense(3,activation='softmax'))

model.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# train the network
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
INIT_LR = 1e-3  #learning rate
EPOCHS = 20     #number of iterations
BS = 64     #batch size
print("[INFO] training network...")
H = model.fit(x=X_train,y=y_t,validation_data=(X_test,y_ts),batch_size=BS,epochs=EPOCHS,callbacks=[es],)


[INFO] training network...
Epoch 1/20
110/110 [==============================] - 3s 21ms/step - loss: 1.0101 - accuracy: 0.5002 - val_loss: 0.9883 - val_accuracy: 0.5160
Epoch 2/20
110/110 [==============================] - 2s 18ms/step - loss: 0.9553 - accuracy: 0.5403 - val_loss: 0.9744 - val_accuracy: 0.5210
Epoch 3/20
110/110 [==============================] - 2s 19ms/step - loss: 0.9238 - accuracy: 0.5605 - val_loss: 0.9670 - val_accuracy: 0.5356
Epoch 4/20
110/110 [==============================] - 2s 18ms/step - loss: 0.8938 - accuracy: 0.5751 - val_loss: 1.0022 - val_accuracy: 0.5422
Epoch 5/20
110/110 [==============================] - 2s 18ms/step - loss: 0.8696 - accuracy: 0.5899 - val_loss: 0.9902 - val_accuracy: 0.5276
Epoch 6/20
110/110 [==============================] - 2s 18ms/step - loss: 0.8351 - accuracy: 0.6083 - val_loss: 1.0246 - val_accuracy: 0.5283
Epoch 7/20
110/110 [==============================] - 2s 18ms/step - loss: 0.8068 - accuracy: 0.6127 - val_loss: 1.

In [36]:

print("[INFO] evaluating network...")
predIdxs = model.predict(X_test, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
y_test_ = np.argmax(y_ts, axis=1)

cm = confusion_matrix(y_test_, predIdxs)

print(cm)
print("acc: {:.2f}".format(np.trace(cm)/sum(sum(cm))))

[INFO] evaluating network...
[[ 241  178  290]
 [ 167  215  438]
 [ 183  225 1069]]
acc: 0.51


**LSTM**

In [47]:
# Design Neural Network Architecture with SimpleRNN
def data_reshape(data):
  return np.reshape(data, (data.shape[0], data.shape[1], 1))
dropout_rate = 0.5
X_train = data_reshape(np.array(X_train))
print('Building Simple RNN Model..')

RNN_model = Sequential()

# Add Simple LSTM layer
RNN_model.add(LSTM(units = 64 , dropout=0.2, recurrent_dropout=0.2, input_shape=(X_train.shape[1], X_train.shape[2])))
# Add Dense Hidden Layer
RNN_model.add(Dense(512, activation='relu'))
RNN_model.add(Dropout(dropout_rate))
RNN_model.add(Dense(256, activation='relu'))
RNN_model.add(Dropout(dropout_rate))
RNN_model.add(Dense(128, activation='relu'))
RNN_model.add(Dropout(dropout_rate))
RNN_model.add(Dense(64, activation='relu'))
RNN_model.add(Dropout(dropout_rate))
# Output Layer 
RNN_model.add(Dense(3,activation='softmax'))

Building Simple RNN Model..


In [48]:
# Configure model
RNN_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train!
print('Training the model..')
history = RNN_model.fit(X_train, y_t, batch_size=BS, verbose=2, epochs=EPOCHS, validation_data=(X_test, y_ts))

Training the model..
Epoch 1/20
110/110 - 45s - loss: 1.0576 - accuracy: 0.4895 - val_loss: 1.0450 - val_accuracy: 0.4914 - 45s/epoch - 412ms/step
Epoch 2/20
110/110 - 42s - loss: 1.0486 - accuracy: 0.4928 - val_loss: 1.0460 - val_accuracy: 0.4914 - 42s/epoch - 384ms/step
Epoch 3/20
110/110 - 42s - loss: 1.0470 - accuracy: 0.4928 - val_loss: 1.0441 - val_accuracy: 0.4914 - 42s/epoch - 384ms/step
Epoch 4/20
110/110 - 42s - loss: 1.0461 - accuracy: 0.4928 - val_loss: 1.0440 - val_accuracy: 0.4914 - 42s/epoch - 385ms/step
Epoch 5/20
110/110 - 42s - loss: 1.0476 - accuracy: 0.4928 - val_loss: 1.0440 - val_accuracy: 0.4914 - 42s/epoch - 384ms/step
Epoch 6/20
110/110 - 42s - loss: 1.0452 - accuracy: 0.4928 - val_loss: 1.0439 - val_accuracy: 0.4914 - 42s/epoch - 385ms/step
Epoch 7/20
110/110 - 42s - loss: 1.0457 - accuracy: 0.4928 - val_loss: 1.0438 - val_accuracy: 0.4914 - 42s/epoch - 384ms/step
Epoch 8/20
110/110 - 42s - loss: 1.0453 - accuracy: 0.4925 - val_loss: 1.0440 - val_accuracy: 0.4

In [60]:
# evaluate the classification model
_,acc = RNN_model.evaluate(X_test, y_ts)
print("acc: %.2f" % (acc))

94/94 [==============================] - 4s 42ms/step - loss: 1.0443 - accuracy: 0.4914
acc: 0.49


**Group matches predictions**

In [53]:
# List for storing the group stage games
pred_set = []



In [54]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Home Team'].map(ranking.set_index('Team')['Positions']))
fixtures.insert(2, 'second_position', fixtures['Away Team'].map(ranking.set_index('Team')['Positions']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:36, :]
fixtures.tail()

ValueError: ignored

In [55]:
fixtures

,Group,first_position,second_position,Home Team,Away Team
0,A,50,60,Cameroon,Burkina Faso
1,A,137,73,Ethiopia,Cape Verde
2,B,20,121,Senegal,Zimbabwe
3,B,81,129,Guinea,Malawi
4,C,28,52,Morocco,Ghana
5,C,132,89,Comoros,Gabon
6,E,29,108,Algeria,Sierra Leone
7,D,36,45,Nigeria,Egypt
8,D,125,106,Sudan,Guinea-Bissau
9,F,30,53,Tunisia,Mali


In [56]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'home_team': row['Home Team'], 'away_team': row['Away Team'], 'winning_team': None})
    else:
        pred_set.append({'home_team': row['Away Team'], 'away_team': row['Home Team'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set

pred_set.head()

,home_team,away_team,winning_team
0,Cameroon,Burkina Faso,None
1,Cape Verde,Ethiopia,None
2,Senegal,Zimbabwe,None
3,Guinea,Malawi,None
4,Morocco,Ghana,None


In [57]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]

# Remove winning team column
pred_set = pred_set.drop(['winning_team'], axis=1)

pred_set.head()

,home_team_Albania,home_team_Algeria,home_team_Andalusia,home_team_Andorra,home_team_Angola,home_team_Argentina,home_team_Armenia,home_team_Australia,home_team_Austria,home_team_Bahrain,home_team_Bangladesh,home_team_Basque Country,home_team_Belarus,home_team_Belgium,home_team_Benin,home_team_Bolivia,home_team_Bosnia and Herzegovina,home_team_Botswana,home_team_Brazil,home_team_Brittany,home_team_Bulgaria,home_team_Burkina Faso,home_team_Burundi,home_team_Cameroon,home_team_Canada,home_team_Cape Verde,home_team_Catalonia,home_team_Central African Republic,home_team_Chad,home_team_Chile,home_team_China PR,home_team_Colombia,home_team_Comoros,home_team_Congo,home_team_Corsica,home_team_Costa Rica,home_team_Croatia,home_team_Cuba,home_team_Czech Republic,home_team_Czechoslovakia,...,away_team_Rwanda,away_team_Réunion,away_team_Saudi Arabia,away_team_Scotland,away_team_Senegal,away_team_Serbia,away_team_Seychelles,away_team_Sierra Leone,away_team_Slovakia,away_team_Slovenia,away_team_Somalia,away_team_South Africa,away_team_South Korea,away_team_South Sudan,away_team_Spain,away_team_Sudan,away_team_Suriname,away_team_Sweden,away_team_Switzerland,away_team_Syria,away_team_São Tomé and Príncipe,away_team_Tanzania,away_team_Thailand,away_team_Togo,away_team_Trinidad and Tobago,away_team_Tunisia,away_team_Turkey,away_team_Uganda,away_team_United Arab Emirates,away_team_United States,away_team_Uruguay,away_team_Uzbekistan,away_team_Venezuela,away_team_Wales,away_team_Yemen,away_team_Yemen DPR,away_team_Yugoslavia,away_team_Zambia,away_team_Zanzibar,away_team_Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [97]:
# Logistic Regression
# group matches 
def predict_logreg(pred_set,backup_pred_set,n):
  predictions = logreg.predict(pred_set)
  for i in range(n):
      print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
      if predictions[i] == 2:
          print("Winner: " + backup_pred_set.iloc[i, 0])
      elif predictions[i] == 1:
          print("Draw")
      elif predictions[i] == 0:
          print("Winner: " + backup_pred_set.iloc[i, 1])
      print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set)[i][2]))
      print('Probability of Draw: ', '%.3f'%(logreg.predict_proba(pred_set)[i][1]))
      print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set)[i][0]))
      print("")

In [98]:
predict_logreg(pred_set,backup_pred_set,fixtures.shape[0])

Burkina Faso and Cameroon
Winner: Cameroon
Probability of Cameroon winning:  0.639
Probability of Draw:  0.255
Probability of Burkina Faso winning:  0.106

Ethiopia and Cape Verde
Winner: Cape Verde
Probability of Cape Verde winning:  0.649
Probability of Draw:  0.134
Probability of Ethiopia winning:  0.217

Zimbabwe and Senegal
Winner: Senegal
Probability of Senegal winning:  0.573
Probability of Draw:  0.259
Probability of Zimbabwe winning:  0.168

Malawi and Guinea
Winner: Guinea
Probability of Guinea winning:  0.598
Probability of Draw:  0.263
Probability of Malawi winning:  0.139

Ghana and Morocco
Winner: Morocco
Probability of Morocco winning:  0.500
Probability of Draw:  0.286
Probability of Ghana winning:  0.214

Comoros and Gabon
Winner: Gabon
Probability of Gabon winning:  0.620
Probability of Draw:  0.219
Probability of Comoros winning:  0.161

Sierra Leone and Algeria
Winner: Algeria
Probability of Algeria winning:  0.675
Probability of Draw:  0.198
Probability of Sierra L

In [103]:
# DNN
#group matches 
def predict_DNN(pred_set, backup_pred_set, n):
  predictions = model.predict(pred_set)
  proba = predictions
  predictions = np.argmax(predictions, axis=1)
  #print(proba)
  #print(predictions)
  for i in range(n):
      print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
      if predictions[i] == 2:
          print("Winner: " + backup_pred_set.iloc[i, 0])
      elif predictions[i] == 1:
          print("Draw")
      elif predictions[i] == 0:
          print("Winner: " + backup_pred_set.iloc[i, 1])
      print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(proba[i][2]))
      print('Probability of Draw: ', '%.3f'%(proba[i][1]))
      print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ', '%.3f'%(proba[i][0]))
      print("")

In [105]:
predict_DNN(pred_set, backup_pred_set, fixtures.shape[0])

Burkina Faso and Cameroon
Winner: Burkina Faso
Probability of Cameroon winning:  0.383
Probability of Draw:  0.168
Probability of Burkina Faso winning:  0.449

Ethiopia and Cape Verde
Winner: Cape Verde
Probability of Cape Verde winning:  0.735
Probability of Draw:  0.053
Probability of Ethiopia winning:  0.213

Zimbabwe and Senegal
Winner: Senegal
Probability of Senegal winning:  0.903
Probability of Draw:  0.090
Probability of Zimbabwe winning:  0.007

Malawi and Guinea
Draw
Probability of Guinea winning:  0.190
Probability of Draw:  0.782
Probability of Malawi winning:  0.028

Ghana and Morocco
Winner: Morocco
Probability of Morocco winning:  0.639
Probability of Draw:  0.347
Probability of Ghana winning:  0.014

Comoros and Gabon
Winner: Gabon
Probability of Gabon winning:  0.859
Probability of Draw:  0.134
Probability of Comoros winning:  0.007

Sierra Leone and Algeria
Winner: Algeria
Probability of Algeria winning:  0.774
Probability of Draw:  0.221
Probability of Sierra Leone w

**round of 16 predictions**

In [106]:
#round of 16 matches
group_16 = [('Burkina Faso', 'Ghana'),
            ('Nigeria', 'Zimbabwe'),
            ('Senegal', 'Cape Verde'),
            ('Tunisia', 'Ivory Coast'),
            ('Guinea', 'Mali'),
            ('Cameroon', 'Gabon'),
            ('Algeria', 'Egypt'),
            ('Morocco', 'Sierra Leone')]


In [107]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to FIFA ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Positions'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Positions'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better, he will be the 'home' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'home_team': matches[j][0], 'away_team': matches[j][1]})
        else:
            dict1.update({'home_team': matches[j][1], 'away_team': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1

    # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    # Remove winning team column
    pred_set = pred_set.drop(['winning_team'], axis=1)

    # Predict!
    print('Logistic Regression Model......................................')
    predict_logreg(pred_set, backup_pred_set, len(pred_set))
    print('DNN Model......................................................')
    predict_DNN(pred_set, backup_pred_set, len(pred_set))
    '''
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 2:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        elif predictions[i] == 1:
            print("Draw")
        elif predictions[i] == 0:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ' , '%.3f'%(logreg.predict_proba(pred_set)[i][0]))
        print('Probability of Draw: ', '%.3f'%(logreg.predict_proba(pred_set)[i][1])) 
        print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set)[i][2]))
        print("")
    '''

In [108]:
clean_and_predict(group_16, ranking, final, logreg)

Logistic Regression Model......................................
Burkina Faso and Ghana
Winner: Ghana
Probability of Ghana winning:  0.691
Probability of Draw:  0.215
Probability of Burkina Faso winning:  0.095

Zimbabwe and Nigeria
Winner: Nigeria
Probability of Nigeria winning:  0.678
Probability of Draw:  0.218
Probability of Zimbabwe winning:  0.103

Cape Verde and Senegal
Winner: Senegal
Probability of Senegal winning:  0.696
Probability of Draw:  0.197
Probability of Cape Verde winning:  0.107

Ivory Coast and Tunisia
Winner: Tunisia
Probability of Tunisia winning:  0.418
Probability of Draw:  0.306
Probability of Ivory Coast winning:  0.276

Guinea and Mali
Winner: Mali
Probability of Mali winning:  0.483
Probability of Draw:  0.337
Probability of Guinea winning:  0.179

Gabon and Cameroon
Winner: Cameroon
Probability of Cameroon winning:  0.666
Probability of Draw:  0.197
Probability of Gabon winning:  0.138

Egypt and Algeria
Winner: Algeria
Probability of Algeria winning:  0.4



**quarter-final predictions**



In [109]:
#quarter final matches
quarters = [('Ghana', 'Nigeria'),
            ('Senegal', 'Tunisia'),
            ('Mali', 'Cameroon'),
            ('Algeria', 'Morocco')]

In [110]:
clean_and_predict(quarters, ranking, final, logreg)

Logistic Regression Model......................................
Ghana and Nigeria
Winner: Nigeria
Probability of Nigeria winning:  0.546
Probability of Draw:  0.253
Probability of Ghana winning:  0.200

Tunisia and Senegal
Winner: Senegal
Probability of Senegal winning:  0.474
Probability of Draw:  0.319
Probability of Tunisia winning:  0.206

Mali and Cameroon
Winner: Cameroon
Probability of Cameroon winning:  0.615
Probability of Draw:  0.225
Probability of Mali winning:  0.161

Algeria and Morocco
Winner: Morocco
Probability of Morocco winning:  0.535
Probability of Draw:  0.298
Probability of Algeria winning:  0.167

DNN Model......................................................
Ghana and Nigeria
Draw
Probability of Nigeria winning:  0.288
Probability of Draw:  0.388
Probability of Ghana winning:  0.324

Tunisia and Senegal
Winner: Senegal
Probability of Senegal winning:  0.441
Probability of Draw:  0.383
Probability of Tunisia winning:  0.177

Mali and Cameroon
Draw
Probability o

**Semi-Final predictions**

In [111]:
#semi final matches
semi = [('Nigeria', 'Senegal'),
        ('Cameroon', 'Morocco')]

In [112]:
clean_and_predict(semi, ranking, final, logreg)

Logistic Regression Model......................................
Nigeria and Senegal
Winner: Senegal
Probability of Senegal winning:  0.425
Probability of Draw:  0.308
Probability of Nigeria winning:  0.267

Cameroon and Morocco
Winner: Morocco
Probability of Morocco winning:  0.443
Probability of Draw:  0.384
Probability of Cameroon winning:  0.174

DNN Model......................................................
Nigeria and Senegal
Winner: Nigeria
Probability of Senegal winning:  0.299
Probability of Draw:  0.227
Probability of Nigeria winning:  0.474

Cameroon and Morocco
Draw
Probability of Morocco winning:  0.096
Probability of Draw:  0.501
Probability of Cameroon winning:  0.403



In [113]:
Prob_Senegal = (0.425 + 0.299) / 2
Prob_Nigeria = (0.267 + 0.474) / 2
Prob_Cameroon = (0.174 + 0.403) / 2
Prob_Morocco = (0.096 + 0.443) / 2
print(Prob_Senegal)
print(Prob_Nigeria)
print(Prob_Cameroon)
print(Prob_Morocco)

0.362
0.3705
0.2885
0.2695


**Final predictions**

In [114]:
# Finals
finals = [('Nigeria', 'Cameroon')]

In [115]:
clean_and_predict(finals, ranking, final, logreg)

Logistic Regression Model......................................
Cameroon and Nigeria
Winner: Nigeria
Probability of Nigeria winning:  0.490
Probability of Draw:  0.345
Probability of Cameroon winning:  0.165

DNN Model......................................................
Cameroon and Nigeria
Winner: Nigeria
Probability of Nigeria winning:  0.588
Probability of Draw:  0.297
Probability of Cameroon winning:  0.115

